In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import json

def calculate_distance(feature_vector1, feature_vector2):
    distance = np.linalg.norm(feature_vector1 - feature_vector2)
    return distance

def find_nearest_neighbors(df, mutation_of_interest):
    # Calculate distances between the mutation of interest and all other mutations
    distances = [(idx, calculate_distance(mutation_of_interest['Normalized Base Pair Position'], row['Normalized Base Pair Position'])) for idx, row in df.iterrows()]

    # Filter distances that have smaller base pair locations than the mutation of interest
    valid_distances = [(idx, dist) for idx, dist in distances if idx < mutation_of_interest.name]

    # If there aren't enough preceding neighbors, return None
    if len(valid_distances) < 2:
        return None, None

    # Sort valid distances in ascending order
    sorted_distances = sorted(valid_distances, key=lambda x: x[1])

    # Find the indices of n1 and n2
    n1_index = sorted_distances[0][0]
    n2_index = sorted_distances[1][0]

    # Retrieve the nearest neighbors from the DataFrame
    n1 = df.loc[n1_index]
    n2 = df.loc[n2_index]

    return n1, n2

def store_triplets(folder_path):
    """
    Iterate through all .pkl files in the given folder,
    load each DataFrame, and store the triplets of samples training/testing.
    """
    all_triplets = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pkl"):
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, 'rb') as f:
                df = pickle.load(f)

            # Drop unnecessary columns
            df = df[['Chromosome', 'Normalized Base Pair Position', 'Mutation Encoded', 'Consequence']]

            for index, row in df.iterrows():
                mutation_of_interest = row
                nearest_neighbors = find_nearest_neighbors(df, mutation_of_interest)
                if nearest_neighbors[0] is not None and nearest_neighbors[1] is not None:
                    n1, n2, n3 = nearest_neighbors[0], nearest_neighbors[1], mutation_of_interest
                    # Ensure n1 < n2 < mutation of interest (so the data can be sequentially processed)
                    if n1['Normalized Base Pair Position'] >= n2['Normalized Base Pair Position']:
                        n1, n2 = n2, n1
                    if n2['Normalized Base Pair Position'] >= n3['Normalized Base Pair Position']:
                        n2, n3 = n3, n2
                    if n1['Normalized Base Pair Position'] >= n2['Normalized Base Pair Position']:
                        n1, n2 = n2, n1
                    if n2['Normalized Base Pair Position'] >= n3['Normalized Base Pair Position']:
                        n2, n3 = n3, n2
                    all_triplets.append((n1, n2, n3))

    return all_triplets

def serialize_triplets(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, pd.Series):
        return obj.to_dict()
    else:
        raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

def write_triplets_to_pickle(triplets, file_path):
    print("Started writing triplets data into a pickle file")
    with open(file_path, "wb") as fp:
        pickle.dump(triplets, fp)
    print("Done writing pickle data into .pkl file")

def read_triplets_from_pickle(file_path):
    print("Started reading triplets data from a pickle file")
    with open(file_path, "rb") as fp:
        triplets = pickle.load(fp)
    print("Done reading triplets data from .pkl file")
    return triplets


triplet_file_path = '/content/drive/MyDrive/DNA_Triplets/triplets.pkl'

# Example usage
folder_path = "/content/drive/MyDrive/NLP_DNA_PKLs/"
triplets = store_triplets(folder_path)
write_triplets_to_pickle(triplets, triplet_file_path)
# Read triplets from JSON file
triplets_from_file = read_triplets_from_pickle("/content/drive/MyDrive/DNA_Triplets/triplets.pkl")
print('Triplets:', triplets_from_file)

Streaming output truncated to the last 5000 lines.
Normalized Base Pair Position                                             0.349969
Mutation Encoded                 [1, 0, -1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,...
Consequence                                                                      7
Name: 17, dtype: object, Chromosome                                                                  001100
Normalized Base Pair Position                                             0.364233
Mutation Encoded                 [0, 1, 0, -1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,...
Consequence                                                                      5
Name: 0, dtype: object), (Chromosome                                                                  001100
Normalized Base Pair Position                                             0.349969
Mutation Encoded                 [1, 0, -1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,...
Consequence                                                         